# Acquire the Data

## Data Sources

The idea of the cars project is to create a small dataset of cars available in India, with the intention to comparing prices, specs, fuel economy etc. across car types. 

- **[SIAM (Society of Indian Automobile Manufacturers)](http://www.siamindia.com/)**: This is the manufacturer government facing association and they do have statistical data on sales and market share of cars by segment and category (though it is paid). They do have one pdf document on fuel economy by car type and engine capacity - [SIAM Fuel Economy Data](http://www.siamindia.com/uploads/filemanager/256th-4W-FE-Data-Declaration.pdf)

- **[ARAI (Automobile Research Association of India)](https://www.araiindia.com/)** - This is the government insitute responsible for providing certification and testing services and I would have expected them to have good database of specs and fuel economy for all the cars. Unfortunately, they don't have any public available data. A website called [ARAI mileage](http://araimileage.in/) seems to have a collected data but in a pdf very similiar to the SIAM format.

- **Car comparison websites** - There are so many car comparison website avaiable - carwale.com, cardekho.com, overdrive.in, cartrade.com, zigwheels.com, auto.ndtv.com but most of them are setup to compare or select 3-4 cars at a time. We actually need a way to scrape the specification data for all the cars. Lets use one of these sites - [Car Zoom](http://carzoom.in/car-specification/) which has good list of specs and price info.

## Scraping Data

Lets start with getting all the car links out of the table pages. There are 11 pages so we will need to cycle through them to get the data.

In [1]:
library(rvest)
library(plyr)

Loading required package: xml2


In [2]:
baseurl <- "http://carzoom.in/car-specification/page/"

In [3]:
cars.link <- character(0)

In [7]:
# Write a function to get links on one page
get_links <- function(pagenum){
  url <- paste0(baseurl, pagenum)
  pg.links <-  read_html(url) %>%
               html_nodes(".car-model li a")%>%
               html_attr("href")
  pg.links
}

In [8]:
get_links(1)

[1] "http://carzoom.in/car-specification/ashok-leyland-stile-le-8-str-diesel/"                   
 [2] "http://carzoom.in/car-specification/ashok-leyland-stile-ls-8-str-diesel/"                   
 [3] "http://carzoom.in/car-specification/ashok-leyland-stile-lx-8-str-diesel/"                   
 [4] "http://carzoom.in/car-specification/ashok-leyland-stile-ls-7-str-diesel/"                   
 [5] "http://carzoom.in/car-specification/ashok-leyland-stile-ls-7-str-alloy-diesel/"             
 [6] "http://carzoom.in/car-specification/ashok-leyland-stile-lx-7-str-diesel/"                   
 [7] "http://carzoom.in/car-specification/ashok-leyland-stile-lx-7-str-alloy-diesel/"             
 [8] "http://carzoom.in/car-specification/aston-martin-rapide-luxe-petrol/"                       
 [9] "http://carzoom.in/car-specification/aston-martin-rapide-s-petrol/"                          
[10] "http://carzoom.in/car-specification/aston-martin-v12-vantage-coupe-petrol/"                 
[11] "http://carzoom.in/car-specification/aston-martin-v8-vantage-coupe-petrol/"                  
[12] "http://carzoom.in/car-specification/aston-martin-v8-vantage-s-coupe-petrol/"                
[13] "http://carzoom.in/car-specification/aston-martin-v8-vantage-roadster-petrol/"               
[14] "http://carzoom.in/car-specification/aston-martin-vanquish-v12-petrol/"                      
[15] "http://carzoom.in/car-specification/2016-audi-a8-l-50-tdi-quattro-premium-plus/"            
[16] "http://carzoom.in/car-specification/2016-audi-a8-l-50-tdi-quattro/"                         
[17] "http://carzoom.in/car-specification/2016-audi-a8-l-60-tdi-quattro/"                         
[18] "http://carzoom.in/car-specification/2016-audi-a8-l-60-tfsi-quattro/"                        
[19] "http://carzoom.in/car-specification/2016-audi-a8-l-w12-quattro/"                            
[20] "http://carzoom.in/car-specification/2016-audi-a8-l-security/"                               
[21] "http://carzoom.in/car-specification/audi-a3-attraction-35-tdi-diesel/"                      
[22] "http://carzoom.in/car-specification/audi-a3-40-tfsi-premium-petrol/"                        
[23] "http://carzoom.in/car-specification/audi-a3-premium-35-tdi-diesel/"                         
[24] "http://carzoom.in/car-specification/audi-a3-premium-plus-35-tdi-diesel/"                    
[25] "http://carzoom.in/car-specification/audi-a3-technology-35-tdi-diesel/"                      
[26] "http://carzoom.in/car-specification/audi-a3-cabriolet-petrol/"                              
[27] "http://carzoom.in/car-specification/audi-a4-2-0-tdi-diesel/"                                
[28] "http://carzoom.in/car-specification/audi-a4-1-8-tfsi-petrol/"                               
[29] "http://carzoom.in/car-specification/audi-a4-2-0-tdi-multitronic-diesel/"                    
[30] "http://carzoom.in/car-specification/audi-a4-s4-3-0-tfsi-quattro-petrol/"                    
[31] "http://carzoom.in/car-specification/audi-a4-3-0-tdi-quattro-diesel/"                        
[32] "http://carzoom.in/car-specification/audi-a6-2-0-tdi-diesel/"                                
[33] "http://carzoom.in/car-specification/audi-a6-35-tfsi-petrol/"                                
[34] "http://carzoom.in/car-specification/audi-a6-3-0-tfsi-quattro-petrol/"                       
[35] "http://carzoom.in/car-specification/audi-a6-3-0-tdi-quattro-diesel/"                        
[36] "http://carzoom.in/car-specification/audi-a7-sportback-3-0-tdi-diesel/"                      
[37] "http://carzoom.in/car-specification/audi-a8-l-3-0-tdi-quattro-diesel/"                      
[38] "http://carzoom.in/car-specification/audi-a8-l-4-2-tdi-quattro-diesel/"                      
[39] "http://carzoom.in/car-specification/audi-a8-l-4-2-fsi-quattro-petrol/"                      
[40] "http://carzoom.in/car-specification/audi-a8-l-w12-petrol/"                                  
[41] "http://carzoom.in/car-specification

In [9]:
# Cycle through the 11 pages and get all the links
for (i in 1:11) {
    links <- get_links(i)
    cars.link <- c(cars.link, links)
}

In [10]:
cars.link

[1] "http://carzoom.in/car-specification/ashok-leyland-stile-le-8-str-diesel/"                         
  [2] "http://carzoom.in/car-specification/ashok-leyland-stile-ls-8-str-diesel/"                         
  [3] "http://carzoom.in/car-specification/ashok-leyland-stile-lx-8-str-diesel/"                         
  [4] "http://carzoom.in/car-specification/ashok-leyland-stile-ls-7-str-diesel/"                         
  [5] "http://carzoom.in/car-specification/ashok-leyland-stile-ls-7-str-alloy-diesel/"                   
  [6] "http://carzoom.in/car-specification/ashok-leyland-stile-lx-7-str-diesel/"                         
  [7] "http://carzoom.in/car-specification/ashok-leyland-stile-lx-7-str-alloy-diesel/"                   
  [8] "http://carzoom.in/car-specification/aston-martin-rapide-luxe-petrol/"                             
  [9] "http://carzoom.in/car-specification/aston-martin-rapide-s-petrol/"                                
 [10] "http://carzoom.in/car-specification/aston-martin-v12-vantage-coupe-petrol/"                       
 [11] "http://carzoom.in/car-specification/aston-martin-v8-vantage-coupe-petrol/"                        
 [12] "http://carzoom.in/car-specification/aston-martin-v8-vantage-s-coupe-petrol/"                      
 [13] "http://carzoom.in/car-specification/aston-martin-v8-vantage-roadster-petrol/"                     
 [14] "http://carzoom.in/car-specification/aston-martin-vanquish-v12-petrol/"                            
 [15] "http://carzoom.in/car-specification/2016-audi-a8-l-50-tdi-quattro-premium-plus/"                  
 [16] "http://carzoom.in/car-specification/2016-audi-a8-l-50-tdi-quattro/"                               
 [17] "http://carzoom.in/car-specification/2016-audi-a8-l-60-tdi-quattro/"                               
 [18] "http://carzoom.in/car-specification/2016-audi-a8-l-60-tfsi-quattro/"                              
 [19] "http://carzoom.in/car-specification/2016-audi-a8-l-w12-quattro/"                                  
 [20] "http://carzoom.in/car-specification/2016-audi-a8-l-security/"                                     
 [21] "http://carzoom.in/car-specification/audi-a3-attraction-35-tdi-diesel/"                            
 [22] "http://carzoom.in/car-specification/audi-a3-40-tfsi-premium-petrol/"                              
 [23] "http://carzoom.in/car-specification/audi-a3-premium-35-tdi-diesel/"                               
 [24] "http://carzoom.in/car-specification/audi-a3-premium-plus-35-tdi-diesel/"                          
 [25] "http://carzoom.in/car-specification/audi-a3-technology-35-tdi-diesel/"                            
 [26] "http://carzoom.in/car-specification/audi-a3-cabriolet-petrol/"                                    
 [27] "http://carzoom.in/car-specification/audi-a4-2-0-tdi-diesel/"                                      
 [28] "http://carzoom.in/car-specification/audi-a4-1-8-tfsi-petrol/"                                     
 [29] "http://carzoom.in/car-specification/audi-a4-2-0-tdi-multitronic-diesel/"                          
 [30] "http://carzoom.in/car-specification/audi-a4-s4-3-0-tfsi-quattro-petrol/"                          
 [31] "http://carzoom.in/car-specification/audi-a4-3-0-tdi-quattro-diesel/"                              
 [32] "http://carzoom.in/car-specification/audi-a6-2-0-tdi-diesel/"                                      
 [33] "http://carzoom.in/car-specification/audi-a6-35-tfsi-petrol/"                                      
 [34] "http://carzoom.in/car-specification/audi-a6-3-0-tfsi-quattro-petrol/"                             
 [35] "http://carzoom.in/car-specification/audi-a6-3-0-tdi-quattro-diesel/"                              
 [36] "http://carzoom.in/car-specification/audi-a7-sportback-3-0-tdi-diesel/"                            
 [37] "http://carzoom.in/car-specification/audi-a8-l-3-0-tdi-quattro-diesel/"                            
 [38] "http://carzoom.in/car-specification/audi-a8-l-4-2-tdi-quattro-diesel/"   

In [11]:
str(cars.link)

 chr [1:813] "http://carzoom.in/car-specification/ashok-leyland-stile-le-8-str-diesel/" ...


In [ ]:
## write.csv(cars, file = "data/cars.link.csv", row.names = FALSE)

Now lets scrape the specification from all the links. We would like to get the name and details of the model, price, type and all the specifications

In [14]:
# Create a function to get the data on one page
get_data <- function(url) {
    pg.url <- url
    pg.html <- read_html(pg.url)
    
    # Get the name and model from the data
    pg.name <- pg.html %>% 
                html_nodes(".crumbs-home+ a") %>%
                html_text()
    
    pg.model<- pg.html %>% 
                html_nodes(".current") %>%
                html_text()
    
    # Get the Price and Body Type info listed in the brand info field
    # This is a vector of length 2
    pg.info <- pg.html %>% 
                html_nodes(".brand_info_fields") %>%
                html_text()
    pg.info
    
    # Get all the specs listed on the page, this is a vector of length n
    pg.spec <- pg.html %>% 
               html_nodes("div.engine") %>%
               html_text()
    n <- length(pg.spec)
    
    # Create similiar length vector for name, model, url, price and type 
    pg.name.rep <- rep(pg.name, n)
    pg.model.rep <- rep(pg.model, n)
    pg.url.rep <- rep(pg.url, n)
    pg.price.rep <- rep(pg.info[1], n)
    pg.type.rep <- rep(pg.info[2], n)
    
    # Create the data frame for the page data and add to the main database
    pg.data <- data.frame(name = pg.name.rep, 
                          model = pg.model.rep,
                          url = pg.url.rep,
                          price = pg.price.rep,
                          type = pg.type.rep, 
                          spec = pg.spec,
                          stringsAsFactors = FALSE)
}


In [ ]:
# Cycle through all the links and create to the cars datafame
cars.spec.raw <- ldply(cars.link, get_data, .progress = "text")

  |                                                                      |   0%

Warning message:
: 'html' is deprecated.
Use 'read_html' instead.
See help("Deprecated")Warning message:
: 'html' is deprecated.
Use 'read_html' instead.
See help("Deprecated")Warning message:
: 'html' is deprecated.
Use 'read_html' instead.
See help("Deprecated")Warning message:
: 'html' is deprecated.
Use 'read_html' instead.
See help("Deprecated")

In [ ]:
## write.csv(cars.spec.raw, file = "data/cars.spec.raw.csv", row.names = FALSE)